In [32]:
import pandas as pd

### 1. concat 데이터 불러오기

17487 rows × 140 columns

In [33]:
df = pd.read_csv('../dataset_kosdaq/ksq_konex_concat.csv',index_col=0)

In [34]:
df = df[df['회계년도'].str.endswith('12')]

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17485 entries, (주)CMG제약 to 한국피아이엠(주)
Columns: 145 entries, 거래소코드 to 이윤분배율
dtypes: float64(131), int64(5), object(9)
memory usage: 19.5+ MB


### 2. 재무비율 생성을 위해 2년 연속 재무제표가 존재하지 않으면 제거

17456 rows × 140 columns

In [36]:
# 회계년도를 시간 데이터로 변경
df['회계년도'] = pd.to_datetime(df['회계년도'], format='%Y/%m')

# 거래소코드와 회계년도 기준으로 정렬
df = df.sort_values(by=['거래소코드', '회계년도'])

# 3년 연속 데이터 체크
def has_consecutive_years(group):
    # year 만 가져오기
    years = group['회계년도'].dt.year
    # 현재 year에서 전 yeear을 뺀값이 1, 총합이 2이상인 것을 반환
    return (years.diff().fillna(1) == 1).cumsum().max() >= 2

# 거래소코드를 기준으로 groupby 후 filter
df = df.groupby('거래소코드').filter(has_consecutive_years)

### 3. Labeling 진행
상장폐지사유 데이터 불러오기

In [37]:
# 회계년도를 datetime
df['회계년도'] = pd.to_datetime(df['회계년도'], format='%Y-%m-%d')
df = df.sort_values(by=['거래소코드', '회계년도'])

# 감사의견코드 중 DI, DS DU, AG, QS가 존재하면 A 아니면 a
df['감사의견_y'] = df['감사의견코드'].apply(lambda x: "A" if x in ['DI', 'DS', 'DU', 'AG', 'QS'] else "a")
# 회계년도를 datetime
df['회계년도'] = pd.to_datetime(df['회계년도'], format='%Y-%m-%d')
df = df.sort_values(by=['거래소코드', '회계년도'])

df['자본잠식률_100'] = df['자본잠식률'].apply(lambda x: "B" if x >= 100 else "b")
# 데이터셋을 거래소코드와 회계년도 기준으로 정렬
df.sort_values(by=['거래소코드', '회계년도'], inplace=True)

# 새로운 'Label' 컬럼을 기본값 'c'로 추가
df['자본잠식률_50'] = 'c'

# 이전 행의 값 초기화
prev_code = None
prev_cap_erode_rate = None

# 각 행을 순회하며 조건에 따라 'Label' 컬럼 설정
for index, row in df.iterrows():
    current_code = row['거래소코드']
    current_cap_erode_rate = row['자본잠식률']

    # 현재 행과 이전 행이 같은 회사이고, 두 해 모두 자본잠식률이 50 이상인 경우 'C'로 표시
    if current_code == prev_code and current_cap_erode_rate >= 50 and prev_cap_erode_rate >= 50:
        df.at[index, '자본잠식률_50'] = 'C'

    # 이전 행 값 업데이트
    prev_code = current_code
    prev_cap_erode_rate = current_cap_erode_rate

# 라벨링 조건에 따라 'y' 컬럼 생성
df['y'] = ((df['감사의견_y'] == 'A') | 
        (df['자본잠식률_100'] == 'B') | 
        (df['자본잠식률_50'] == 'C')).astype(int)

In [38]:
label = pd.read_csv('../dataset_kosdaq/상장폐지사유.csv')

In [39]:
label_list = ['감사의견 거절(감사범위 제한)','상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사유 미해소', "상장폐지 신청('23.06.28)", '감사의견 거절(감사범위 제한 및 계속기업가정 불확실성)',
    '기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정',
    '감사의견 거절(감사범위 제한 및 계속기업 존속능력 불확실성)',
    '기업의 계속성 및 경영의 투명성 등을 종합적을 고려하여 상장폐지기준에 해당한다고 결정',
    '상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동사유 미해소', '감사의견 거절',
    '감사의견 거절(감사범위 제한 및 계속기업 존속능력에 대한 불확실성)',
    '발행한 어음 또는 수표가 주거래은행에 의하여 최종부도로 결정되거나 거래은행에 의한 거래정지',
    '해산사유 발생(파산선고)', '감사의견 한정(감사범위 제한)',
    '법정제출기한까지 사업보고서를 제출하지 아니한 후, 법정제출기한의 다음날부터 10일이내에 사업보고서를 제출하지 아니함',
    '기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당',
    '최근 2년간 3회 이상 공시규정 제19조제1항의 규정에 의한 사업보고서, 반기보고서 또는 분기보고서 법정제출기한 내 미제출',
    '감사의견거절(감사범위 제한)', '감사의견거절(감사범위제한)',
    '감사의견거절(감사범위제한 및 계속기업 존속 불확실)',
    '제28조제1항제9호에 따라 관리종목 지정 후 공시규정 제19조제1항에 따른 사업보고서 법정제출기한 내 미제출, 최근 2년간 3회 이상 공시규정 제19조제1항의 규정에 의한 사업보고서, 반기보고서 또는 분기보고서 법정제출기한 내 미제출',
    '상장예비심사 청구서 미제출로 관리종목 지정후 1개월 이내 동사유를 미해소',
    '제28조제1항제9호에 따라 관리종목 지정 후 공시규정 제19조 제1항에 따른 분기보고서 법정기한 내 미제출',
    '감사의견거절(감사범위 제한 및 계속기업 존속능력에 대한 불확실성)',
    '감사의견거절(범위제한 및 계속기업 존속능력에 대한 불확실성)',
    "'16사업연도 감사의견거절(계속기업 존속능력에 대한 불확실성) 및 '17사업연도 반기 감사의견거절(계속기업 존속능력에 대한 불확실성)",
    '감사의견 거절(내부회계관리제도상의 취약점, 계속기업가정의 불확실성 및 재고자산 관련 감사범위 제한)',
    '감사범위제한으로 인한 한정의견', '자본전액잠식 등',
    '최근 5사업연도 연속 영업손실 발생',
    '감사의견 거절(감사범위 제한 및 기업회계기준 위배)', '2반기 연속 자본잠식률 50% 이상',
    '감사의견 거절(감사범위 제한으로 인한 의견거절)',
    "'14사업연도 자본잠식률 50%이상 사유로 관리종목 지정 후 '15사업연도 반기 감사의견 거절", '자본전액잠식',
    '기업의 계속성, 경영의 투명성 또는 그 밖에 코스닥시장의 건전성 등을 종합적으로 고려하여 상장폐지 기준에 해당한다고 결정',
    '감사의견 거절(계속기업 존속능력에 대한 불확실성 및 감사범위 제한)',
    '발행한 어음 또는 수표가 주거래은행에 의하여 최종부도로 결정되거나 거래은행에 의한 거래 정지',
    "상장폐지 신청('15.01.15)", '감사의견 거절(계속기업 존속능력에 대한 불확실성)',
    '감사의견 거절(감사범위 제한 및 계속기업 존속능력에 대한 불확실성', "상장폐지신청('14.04.22)",
    '최종부도 발생', '자본전액잠식등', '최근 5사업연도 연속 영업손실 발생 등',
    '기업의 계속성, 경영의 투명성 또는 기타 코스닥시장의 건전성 등을 종합적으로 고려하여 상장폐지가 필요하다고 인정',
    "상장폐지신청('13.09.06)", '의견거절(감사범위 제한)',
    '감사의견거절(감사범위 제한 및 계속기업으로서의 존속능력에 대한 불확실성)', "상장폐지 신청('13.05.08)",
    '감사의견거절(계속기업으로서의 존속능력에 대한 불확실성)', '감사의견 부적정',
    '반기 재무제표에 대한 검토의견거절로 관리종목 지정후 자본잠식률 50% 이상', '2반기 연속 자본잠식률 50%이상',
    '상장예비심사 청구서 미제출로 관리종목 지정후 1개월 이내 동 사유 미해소','외부감사인의 감사의견 의견거절',
    '발행한 어음 또는 수표가 주거래은행에 의하여 최종부도로 결정되거나 거래은행에 의한 거래정지',
    '감사의견 의견거절(감사범위제한 및 계속기업 존속불확실성)', 
    '외부감사인의 감사의견 거절(감사범위 제한), 감사의견 한정(감사범위 제한)',
    '외부감사인의 감사의견 의견거절(감사범위 제한)', '감사의견 거절(계속기업 존속능력에 대한 불확실성)',
    '파산선고에 따른 해산사유 발생',
    '감사의견 거절(감사범위 제한)', 
    '감사의견 거절(감사범위 제한 및 계속기업 존속능력에 대한 불확실성)',
    '최종부도 발생','감사의견 거절(감사범위제한)']

In [40]:
# Filtering the dataset for rows where 폐지사유 is in the label list
filtered_data = label[label['폐지사유'].isin(label_list)]

# Selecting only the 종목코드 and 폐지일자 columns
result = filtered_data[['종목코드', '폐지일자']]


In [41]:
import pandas as pd

# df1에서 '종목코드'를 기준으로 그룹화하고 '회계년도'를 오름차순으로 정렬합니다.
df1_sorted = df.groupby('거래소코드').apply(lambda x: x.sort_values('회계년도', ascending=True)).reset_index(drop=True)

# df2의 '폐지일자'를 datetime 형식으로 변환하고 '폐지년'을 추출합니다.
result['폐지일자'] = pd.to_datetime(result['폐지일자'])
result['폐지년'] = result['폐지일자'].dt.year
result['폐지전년'] = result['폐지년'] - 1  # '폐지년'의 전년도를 계산합니다.

# df1에 '회계년' 컬럼을 추가합니다.
df['회계년'] = pd.to_datetime(df['회계년도']).dt.year

# df1의 각 행에 대해 df2의 '폐지전년'과 일치하는 '회계년'이 있는지 확인하고, 
# 일치하는 경우 해당 행의 'y' 라벨을 1로 설정합니다.
delisting_dict = result.set_index('종목코드')['폐지전년'].to_dict()
df['D'] = df.apply(lambda row: "D" if delisting_dict.get(row['거래소코드'], 0) == row['회계년'] else "d", axis=1)

# Removing the additional columns (except 'y') that were added during the process
columns_to_remove = ['회계년', '폐지전년','폐지일자','폐지년']
df1_cleaned = df.drop(columns=columns_to_remove, errors='ignore')


C:\Users\hotte\AppData\Local\Temp\ipykernel_20656\1715330378.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['폐지일자'] = pd.to_datetime(result['폐지일자'])
C:\Users\hotte\AppData\Local\Temp\ipykernel_20656\1715330378.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['폐지년'] = result['폐지일자'].dt.year
C:\Users\hotte\AppData\Local\Temp\ipykernel_20656\1715330378.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [42]:
df = df1_cleaned

In [43]:
# 라벨링 조건에 따라 'y' 컬럼 생성
df['y'] = ((df['감사의견_y'] == 'A') | 
        (df['자본잠식률_100'] == 'B') | 
        (df['자본잠식률_50'] == 'C') |
        (df['자본잠식률_50'] == 'D')).astype(int)

### 4. 결측치

한 컬럼의 결측치가 존재하면 기업 별로 컬럼의 중앙값으로 대체

In [44]:

# 한글로 주석 처리하여 코드 재작성 및 설명 추가

# 1단계: '거래소코드' 기준으로 데이터 그룹화
grouped = df.groupby('거래소코드')

# 2단계: 'y' 컬럼 값이 1인 '거래소코드' 그룹 찾기
codes_with_y_1 = grouped.filter(lambda x: (x['y'] == 1).any()).groupby('거래소코드')

# '거래소코드' 중 전체 컬럼이 NaN인 경우를 저장할 리스트 초기화
codes_with_all_nan_columns = []

# 3단계, 4단계, 5단계를 위한 반복문
for name, group in codes_with_y_1:
    # NaN 값을 가진 컬럼 식별
    nan_columns = group.columns[group.isna().any()].tolist()

    # 컬럼 유형 확인 및 필요시 NaN을 중앙값으로 대체
    for col in nan_columns:
        if group[col].dtype in ['float64', 'int64']:
            if group[col].isna().all():
                # 모든 값이 NaN인 경우 해당 '거래소코드'를 리스트에 추가
                codes_with_all_nan_columns.append(name)
            else:
                # NaN을 해당 그룹의 해당 컬럼의 중앙값으로 대체
                group[col].fillna(group[col].median(), inplace=True)

# 모든 컬럼이 NaN인 '거래소코드' 목록 출력
codes_with_all_nan_columns


[41060,
 44990,
 44990,
 93510,
 148780,
 150440,
 179720,
 188260,
 199730,
 220110,
 220250,
 222670,
 222670,
 229480,
 229480,
 238170,
 238170,
 240600,
 246830,
 248020,
 253610,
 254160,
 263750,
 266170,
 283100,
 283100,
 284620,
 286000,
 288490,
 298380,
 299910,
 300080,
 308080,
 309900,
 318660,
 318660,
 321370,
 322190,
 329020,
 329050,
 336040,
 337840,
 343090,
 343090,
 343090,
 348840,
 348840,
 351020,
 351020,
 352910,
 355390,
 355390,
 357880,
 358570,
 365270,
 372320,
 372320,
 376300,
 376980,
 377480,
 388870,
 388870,
 397030,
 406820,
 413640,
 413640,
 416180,
 417860,
 418470,
 424960,
 432720,
 432720,
 440110,
 445180,
 446540,
 446600,
 447690,
 450520,
 450520,
 456570,
 456570,
 900040,
 900040,
 900040,
 900040,
 900040,
 900040,
 900040,
 900040,
 900040,
 900040,
 900040,
 900040,
 900040,
 900040,
 900040,
 900040,
 900040,
 900080,
 900080,
 900080,
 900080,
 900080,
 900080,
 900080,
 900080,
 900080,
 900080,
 900080,
 900080,
 900080,
 9000

In [45]:
len(set(codes_with_all_nan_columns))

67

In [46]:
# 거래소코드를 기준으로 그룹바이, 첫행 삭제
grouped_df = df.groupby('거래소코드').apply(lambda x: x.iloc[1:]).reset_index(drop=True)

### 5. 부도년도 이후 데이터 삭제

15246 rows × 150 columns

In [47]:
def remove_post_default_data(group):
    # 최초 부도 년도 설정
    default_year = group[group['y'] == 1]['회계년도'].min()
    
    # default 값이 nan이 아니면 작거나 같은 데이터만 가져옴
    if not pd.isnull(default_year):
        group = group[group['회계년도'] <= default_year]
    
    return group

# 회사명으로 그룹바이 후 함수 적용
df = grouped_df.groupby('거래소코드').apply(remove_post_default_data).reset_index(drop=True)

In [48]:
df[(df['y']==1)&(df['자본잠식률'].isna())]

,거래소코드,회계년도,순이익증가율,자본잠식률,영업이익증가율,총자본증가율,자기자본증가율,유형자산증가율,유동자산증가율,재고자산증가율,...,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,감사의견_y,자본잠식률_100,자본잠식률_50,y,D
9799,149300,2016-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,a,b,C,1,d


### 6. 부도년도 이전 데이터

전년도의 자본잠식률이 50%면 1
나머지 데이터는 삭제

182114 rows × 156 columns

In [49]:
# 거래소 코드와 회계년도를 기준으로 정렬
df = df.sort_values(by=['거래소코드', '회계년도'])

for i in range(1, len(df)):
    # y가 1 인 경우
    if df.loc[i, 'y'] == 1:
        # 해당 행과 전 행의 거래소 코드가 같으면
        if df.loc[i, '거래소코드'] == df.loc[i-1, '거래소코드']:
            # 전 행의 자본잠식률이 50이상이면
            if df.loc[i-1, '자본잠식률'] >= 50:
                # 전 행의 y 가 1
                df.loc[i-1, 'y'] = 1

In [50]:
# 부도 데이터 중 정상 데이터 삭제

def remove_post_default_data(group):
    # 최초 부도 년도 설정
    default_year = group[group['y'] == 1]['회계년도'].min()
    
    # default 값이 nan이 아니면 크거나 같은 데이터만 가져옴
    if not pd.isnull(default_year):
        group = group[group['회계년도'] >= default_year]
    
    return group

# 회사명으로 그룹바이 후 함수 적용
df = df.groupby('거래소코드').apply(remove_post_default_data).reset_index(drop=True)

# 7. 정상기업 1번째 회계년도 제거
# 전체 데이터 회계년도 2012년도 데이터 삭제

In [51]:
df.to_csv('../dataset_kosdaq/상장최종.csv')

In [61]:
df[df['y']==1]['거래소코드'].nunique()

329